In [18]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
from keras.callbacks import ModelCheckpoint
#from keras.applications.resnet50 import preprocess_input
from keras import Model
from keras import initializers
from keras.callbacks import LearningRateScheduler
import resnet
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from classification_models import ResNet18
from classification_models.resnet import preprocess_input

In [36]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
train_batchsize = 64
val_batchsize = 64

train_dir = "D:/retinal_data_set_visioncare/TrainValidationData/augmented_train_data_224"
validation_dir = "D:/retinal_data_set_visioncare/TrainValidationData/validation"
test_dir = "D:/retinal_data_set_visioncare/newTrainValidationTestData/new_test"

#train_dir = "D:/retinal_data_set_visioncare/New_Train_Test_Data/train"
#validation_dir = "D:/retinal_data_set_visioncare/TrainValidationData/validation"
#test_dir = "D:/retinal_data_set_visioncare/New_Train_Test_Data/test"

model_check_point_loc = 'D:/retinal_data_set_visioncare/models/resnet18/resnet_updated_all_cnn_dr.h5'

#resnet_base = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))
#resnet_base = resnet.ResNet101(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))
resnet_base = ResNet18(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))


In [37]:
# Freeze the layers except the last 2 layers
for layer in resnet_base.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in resnet_base.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x000002AB86406EF0> False
<keras.layers.normalization.BatchNormalization object at 0x000002AB86406A20> False
<keras.layers.convolutional.ZeroPadding2D object at 0x000002AB86406CC0> False
<keras.layers.convolutional.Conv2D object at 0x000002AB8642E668> False
<keras.layers.normalization.BatchNormalization object at 0x000002AB86406E80> False
<keras.layers.core.Activation object at 0x000002AB851DC7F0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x000002AB851EEDD8> False
<keras.layers.pooling.MaxPooling2D object at 0x000002AB851DC2B0> False
<keras.layers.normalization.BatchNormalization object at 0x000002AB8527BDD8> False
<keras.layers.core.Activation object at 0x000002AB852D0940> False
<keras.layers.convolutional.ZeroPadding2D object at 0x000002AB852D7B38> False
<keras.layers.convolutional.Conv2D object at 0x000002AB852D0BA8> False
<keras.layers.normalization.BatchNormalization object at 0x000002AB85319208> False
<keras.layers.core.

In [38]:
resnet_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_55 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_55[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

In [39]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_width, image_height),
        batch_size=train_batchsize,
        shuffle=True,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_width, image_height),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(image_width, image_height),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 5000 images belonging to 5 classes.
Found 278 images belonging to 5 classes.
Found 142 images belonging to 5 classes.


In [40]:

# Create the model
# add a global spatial average pooling layer 28 , 28 , 128
x = resnet_base.get_layer(index=-1).output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7, name='global_avg_dropout')(x)
# add a fully-connected layer
# x = Dense(128, kernel_initializer=initializers.he_normal(seed=None), kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01), use_bias=False)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5, name='fc1_dropout')(x)
# x = Dense(7, kernel_initializer=initializers.he_normal(seed=None), use_bias=False, kernel_regularizer=regularizers.l2(0.001))(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5, name='fc2_dropout')(x)
# and a fully connected output/classification layer
#x = Dense(5, kernel_initializer=initializers.he_normal(seed=None))(x)
#x = Dense(5, activity_regularizer=regularizers.l2(0.001), use_bias=False)(x)
#kernel_regularizer=regularizers.l2(0.001), activity_regularizer=regularizers.l2(0.001)
x = Dense(5, kernel_initializer=initializers.he_normal(seed=None), activity_regularizer=regularizers.l2(0.001), use_bias=False)(x)
x = BatchNormalization()(x)
predictions = Activation('softmax')(x)
# create the full network so we can train on it
model = Model(inputs=resnet_base.input, outputs=predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_55 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_55[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

In [68]:
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
# #model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Dropout(0.25))
# model.add(GlobalAveragePooling2D())
# #model.add(Flatten())
# model.add(Dense(15, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(5, activation='softmax'))

# model.summary()
# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.Adadelta(),
#               metrics=['accuracy'])

In [43]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=['acc'])

In [44]:
model_checkpoint = ModelCheckpoint(model_check_point_loc, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

In [10]:
# class_weights_dic = {0: 0.37236180904522614,
#  1: 1.5278350515463917,
#  2: 1.2453781512605042,
#  3: 2.9058823529411764,
#  4: 1.95}

In [11]:
# class_weights_dic
# class_weight=class_weights_dic,

In [45]:
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size,
      epochs=100,
      callbacks=[model_checkpoint],
      validation_data=test_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      verbose=1)

Epoch 1/100
78/78 [==============================] - 74s 943ms/step - loss: 2.6896 - acc: 0.2560 - val_loss: 1.6036 - val_acc: 0.1942
Epoch 2/100
78/78 [==============================] - 68s 869ms/step - loss: 2.0056 - acc: 0.3678 - val_loss: 1.4611 - val_acc: 0.3204
Epoch 3/100
78/78 [==============================] - 68s 874ms/step - loss: 1.6030 - acc: 0.4499 - val_loss: 1.3976 - val_acc: 0.5971
Epoch 4/100
78/78 [==============================] - 68s 876ms/step - loss: 1.3740 - acc: 0.5144 - val_loss: 1.2677 - val_acc: 0.7476
Epoch 5/100
78/78 [==============================] - 69s 884ms/step - loss: 1.2826 - acc: 0.5323 - val_loss: 1.1385 - val_acc: 0.7379
Epoch 6/100
78/78 [==============================] - 69s 886ms/step - loss: 1.2785 - acc: 0.5101 - val_loss: 1.1359 - val_acc: 0.7330
Epoch 7/100
78/78 [==============================] - 69s 888ms/step - loss: 1.2700 - acc: 0.5164 - val_loss: 1.2150 - val_acc: 0.6845
Epoch 8/100
78/78 [==============================] - 68s 875ms

Epoch 62/100
78/78 [==============================] - 69s 883ms/step - loss: 1.1771 - acc: 0.5517 - val_loss: 0.9869 - val_acc: 0.7670
Epoch 63/100
78/78 [==============================] - 68s 878ms/step - loss: 1.2015 - acc: 0.5348 - val_loss: 0.9801 - val_acc: 0.7767
Epoch 64/100
78/78 [==============================] - 68s 871ms/step - loss: 1.1842 - acc: 0.5491 - val_loss: 0.9671 - val_acc: 0.7233
Epoch 65/100
78/78 [==============================] - 67s 861ms/step - loss: 1.1795 - acc: 0.5417 - val_loss: 1.1178 - val_acc: 0.6117
Epoch 66/100
78/78 [==============================] - 69s 879ms/step - loss: 1.1993 - acc: 0.5296 - val_loss: 1.1592 - val_acc: 0.5631
Epoch 67/100
78/78 [==============================] - 68s 870ms/step - loss: 1.1983 - acc: 0.5345 - val_loss: 0.9334 - val_acc: 0.7379
Epoch 68/100
78/78 [==============================] - 68s 872ms/step - loss: 1.1937 - acc: 0.5368 - val_loss: 1.0581 - val_acc: 0.7864
Epoch 69/100
78/78 [==============================] - 6

In [13]:
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size,
      epochs=150,
      callbacks=[model_checkpoint],
      validation_data=test_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      verbose=1)

Epoch 1/150
78/78 [==============================] - 89s 1s/step - loss: 2.8143 - acc: 0.2384 - val_loss: 1.7667 - val_acc: 0.1748
Epoch 2/150
78/78 [==============================] - 69s 891ms/step - loss: 2.1014 - acc: 0.3396 - val_loss: 1.5484 - val_acc: 0.3835
Epoch 3/150
78/78 [==============================] - 69s 883ms/step - loss: 1.6722 - acc: 0.4357 - val_loss: 1.4311 - val_acc: 0.3883
Epoch 4/150
78/78 [==============================] - 68s 871ms/step - loss: 1.4058 - acc: 0.5028 - val_loss: 1.3222 - val_acc: 0.6893
Epoch 5/150
78/78 [==============================] - 69s 880ms/step - loss: 1.3167 - acc: 0.5231 - val_loss: 1.1293 - val_acc: 0.7184
Epoch 6/150
78/78 [==============================] - 68s 872ms/step - loss: 1.2730 - acc: 0.5202 - val_loss: 1.0838 - val_acc: 0.7282
Epoch 7/150
78/78 [==============================] - 69s 879ms/step - loss: 1.2671 - acc: 0.5183 - val_loss: 1.1154 - val_acc: 0.7184
Epoch 8/150
78/78 [==============================] - 69s 883ms/st

KeyboardInterrupt: 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
# Returns a compiled model identical to the previous one
loaded_model = load_model(model_check_point_loc)

In [ ]:
# #Confution Matrix and Classification Report
# Y_pred = loaded_model.predict_generator(validation_generator, validation_generator.samples // validation_generator.batch_size + 1)
# y_pred = np.argmax(Y_pred, axis=1)
# print('Confusion Matrix')
# print(confusion_matrix(validation_generator.classes, y_pred))
# print('Classification Report')
# target_names = ['non-dr', 'mild-npdr', 'moderate-npdr', 'severe-npdr', 'pdr']
# print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
#Confution Matrix and Classification Report for testset
Y_pred_for_test = loaded_model.predict_generator(test_generator, test_generator.samples // test_generator.batch_size)
Y_pred_for_test = np.argmax(Y_pred_for_test, axis=1)
print('Confusion Matrix for testset')
print(confusion_matrix(test_generator.classes, Y_pred_for_test))
print('Classification Report')
target_names = ['non-dr', 'mild-npdr', 'moderate-npdr', 'severe-npdr', 'pdr']
print(classification_report(test_generator.classes, Y_pred_for_test, target_names=target_names))

In [ ]:
# generate roc curve
n_classes = 5

# Plot linewidth.
lw = 2

#convert original class labels to the one-hot-encoding
y_test = np_utils.to_categorical(test_generator.classes, 5)

Y_test_pred = loaded_model.predict_generator(test_generator, test_generator.samples // test_generator.batch_size)
Y_test_predicted = np.argmax(Y_test_pred, axis=1)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], Y_test_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), Y_test_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()
